In [13]:
import torch
import torch.nn as nn
import torch.optim as optim



In [14]:
data_test=  []
lab_test = []
# ones = True
# for i in range(10):
#     tdat  = []
#     tlab = []
    
#     for i in range(90):
#         tdat.append(i)
#         if ones:
#             tlab.append(1)
#         else:
#             tlab.append(0)
#     ones = not ones
#     data_test.append(tdat)
#     lab_test.append(tlab)
# X_train = torch.FloatTensor(data_test)
# X_test = X_train
# y_train = torch.FloatTensor(lab_test)
# y_test = y_train
import numpy as np

# Create an array of length 10, with each element being an array of length 90
data = np.random.rand(10, 90)

# Create labels for each element (assume binary labels)
labels = np.random.randint(0, 2, size=10)

# Split the data into X_train and y_train
X_train = data
y_train = labels
print(X_train)
print(y_train)
# Convert X_train and y_train to PyTorch tensors (if using PyTorch)
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float().unsqueeze(1)
X_train = X_train_tensor
y_train = y_train_tensor


[[0.96119056 0.90535064 0.19579113 0.0693613  0.100778   0.01822183
  0.09444296 0.68300677 0.07118865 0.31897563 0.84487531 0.02327194
  0.81446848 0.28185477 0.11816483 0.69673717 0.62894285 0.87747201
  0.73507104 0.80348093 0.28203457 0.17743954 0.75061475 0.80683474
  0.99050514 0.41261768 0.37201809 0.77641296 0.34080354 0.93075733
  0.85841275 0.42899403 0.75087107 0.75454287 0.10312387 0.90255291
  0.50525237 0.82645747 0.3200496  0.89552323 0.38920168 0.01083765
  0.90538198 0.09128668 0.31931364 0.95006197 0.95060715 0.57343789
  0.63183721 0.44844552 0.29321077 0.32866455 0.67251846 0.75237453
  0.79157904 0.78961814 0.0912061  0.4944203  0.05755876 0.54952888
  0.4415305  0.88770418 0.35091501 0.11706702 0.14299168 0.76151063
  0.61821806 0.10112268 0.08410681 0.70096913 0.07276301 0.82186006
  0.70624223 0.08134878 0.08483771 0.98663958 0.3742708  0.37064215
  0.81279957 0.94724858 0.98600106 0.75337819 0.37625959 0.08350072
  0.77714692 0.55840425 0.42422201 0.90635439 0.

In [15]:
import os 
import glob
import pandas as pd
from torch.utils.data import Dataset
import numpy as np
import torch
from glob import glob

class TimeSeriesDataset(Dataset):
    def __init__(self, folder_nolayoff, folder_layoff, src_col):
        
        # csvs_layoffs = glob.glob(folder_layoff + "/*.csv")
        # csvs_nolayoffs = glob.glob(folder_nolayoff + "/*.csv")
        X_train = []
        Y_train = []
        all_csv_files = []
        EXT = "*.csv"  # Define the variable EXT
        for path, subdir, files in os.walk(folder_layoff):
            
            for file in glob(os.path.join(path, EXT)):
                # print(file)
                df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
                ## If length of dataset is over 90, then cut it to 90
                if len(df) > 90:
                    print("OVER LENGTH", file)
                    df = df[-90:]
                ## If length of dataset is less than 90, then skip it
                if len(df) < 90:
                    continue
                X_train.append(df[src_col].values)
                Y_train.append(float(1))
                # all_csv_files.append(file)

        for path, subdir, files in os.walk(folder_nolayoff):
            for file in glob(os.path.join(path, EXT)):
                df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
                ## If length of dataset is over 90, then cut it to 90
                if len(df) > 90:
                    print("OVER LENGTH", file)
                    df = df[-90:]

                ## If length of dataset is less than 90, then skip it
                if len(df) < 90:
                    continue
                X_train.append(df[src_col].values)
                Y_train.append(float(0))
                # all_csv_files.append(file)





        # for directory in csvs_layoffs:
        #     for csv in os.listdir(directory):
        #         if csv.endswith(".csv"):
        #             df = pd.read_csv(csv, index_col=0, parse_dates=True).sort_index()
        #             X_train.append(df[src_col].values)
        #             Y_train.append(1)
        #     # df = pd.read_csv(csv, index_col=0, parse_dates=True).sort_index()
        #     # X_train.append(df[src_col].values)
        #     # Y_train.append(1)
        # for directory in csvs_nolayoffs:
        #     for csv in os.listdir(directory):
        #         if csv.endswith(".csv"):
        #             df = pd.read_csv(csv, index_col=0, parse_dates=True).sort_index()
        #             X_train.append(df[src_col].values)
        #             Y_train.append(0)
            
        X_train = torch.from_numpy(np.array(X_train))
        Y_train = torch.from_numpy(np.array(Y_train))
        # if len(X_train.shape) < 3:
        #     X_train = X_train.unsqueeze(2)

        # if X_train.shape.index(min(X_train.shape[1], X_train.shape[2])) != 1:  # make sure the Channels in second dim
        #     X_train = X_train.permute(0, 2, 1)

        self.X_train = X_train
        self.Y_train = Y_train
        self.num_channels = X_train.shape[1]
        self.len = X_train.shape[0]

    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        return self.X_train[idx].float(), self.Y_train[idx]



In [68]:
from torch.utils.data.sampler import SubsetRandomSampler

dataset = TimeSeriesDataset("stocks_no_layoffs", "stocks_layoffs", "Open")
batch_size = 1
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

C:\Users\alexs\AppData\Local\Temp\ipykernel_38288\1456045152.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipykernel_38288\1456045152.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipykernel_38288\1456045152.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipy

In [17]:
from torch.utils.data import DataLoader

In [18]:
tdloader = TimeSeriesDataset("stocks_no_layoffs", "stocks_layoffs", "open_in_proportion_to_average")

C:\Users\alexs\AppData\Local\Temp\ipykernel_38288\1456045152.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipykernel_38288\1456045152.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipykernel_38288\1456045152.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipy

In [19]:
layoff_data_loader = DataLoader(dataset=tdloader, batch_size=1, 
                                shuffle=True, drop_last=False, num_workers=0)

In [20]:
layoff_data_loader.dataset.X_train.shape

torch.Size([516, 90])

In [21]:
## Iterate through dataloader and print y labels
for i, (x, y) in enumerate(layoff_data_loader):
    print(y)
    # print(x)
    # break

tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], d

In [69]:
# Define the model architecture
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(90, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Create an instance of the model
model = BinaryClassifier()

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Training loop
num_epochs = 100
batch_size = 32

for epoch in range(num_epochs):
    ## Run through dataloader batches
    



    for i, (X_train, y_train) in enumerate(train_loader):
    
        # Forward pass
        outputs = model(X_train)
        y_train = y_train.unsqueeze(1)
        loss = criterion(outputs.float(), y_train.float())

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# # Evaluation
# with torch.no_grad():
#     outputs = model(X_test)
#     predicted = (outputs >= 0.5).float()
#     accuracy = (predicted == y_test).float().mean()
#     print(f'Test Accuracy: {accuracy:.4f}')

# # Make predictions
# with torch.no_grad():
#     outputs = model(X_new)
#     predicted = (outputs >= 0.5).float()

Epoch [1/100], Loss: 0.5889
Epoch [2/100], Loss: 0.6208
Epoch [3/100], Loss: 0.7308
Epoch [4/100], Loss: 2.0732
Epoch [5/100], Loss: 0.7912
Epoch [6/100], Loss: 0.7181
Epoch [7/100], Loss: 0.2664
Epoch [8/100], Loss: 0.7078
Epoch [9/100], Loss: 0.7307
Epoch [10/100], Loss: 0.5894
Epoch [11/100], Loss: 0.6059
Epoch [12/100], Loss: 0.7471
Epoch [13/100], Loss: 0.7203
Epoch [14/100], Loss: 0.7143
Epoch [15/100], Loss: 0.7757
Epoch [16/100], Loss: 0.5753
Epoch [17/100], Loss: 0.6272
Epoch [18/100], Loss: 0.6485
Epoch [19/100], Loss: 0.6897
Epoch [20/100], Loss: 0.8108
Epoch [21/100], Loss: 0.7129
Epoch [22/100], Loss: 0.1093
Epoch [23/100], Loss: 0.7291
Epoch [24/100], Loss: 1.2490
Epoch [25/100], Loss: 0.5994
Epoch [26/100], Loss: 0.6461
Epoch [27/100], Loss: 0.3095
Epoch [28/100], Loss: 0.8605
Epoch [29/100], Loss: 0.3153
Epoch [30/100], Loss: 0.9137
Epoch [31/100], Loss: 0.6109
Epoch [32/100], Loss: 0.6170
Epoch [33/100], Loss: 0.6157
Epoch [34/100], Loss: 0.5877
Epoch [35/100], Loss: 0

In [70]:
is_correct = []
for i, (X_train, y_train) in enumerate(validation_loader):
        # print(X_train)
        # print(y_train)
        # Forward pass
    outputs = model(X_train)
    out = outputs.squeeze(1)
    output_bin = (out >= 0.5).int().item()
    correct = output_bin == int(y_train.item())
    is_correct.append(correct)


In [71]:
# Count ratio of trues and falses in is_correct
true_count = sum(is_correct)
false_count = len(is_correct) - true_count
print(f"True count: {true_count}")
print(f"False count: {false_count}")
print(f"True ratio: {true_count / len(is_correct)}")

True count: 44
False count: 59
True ratio: 0.42718446601941745
